In [13]:
import os
from zeroae.rocksdb import c as crocksdb

In [14]:
DBPath="/tmp/rocksdb_simple_example"
DBBackupPath="/tmp/rocksdb_simple_example_backup"

### Optimize RocksDB
The easiest way to get RocksDB to perform well -- use all your cores.

In [15]:
options = crocksdb.options.create()
crocksdb.options.increase_parallelism(options,os.cpu_count())
crocksdb.options.optimize_level_style_compaction(options,0)
crocksdb.options.set_create_if_missing(options, 1)

### Open the database

In [16]:
db, err = crocksdb.open(options, DBPath)
assert err is None

### Open the backup engine
We will use it for backing up our database.

In [17]:
be, err = crocksdb.backup_engine.open(options, DBBackupPath);
assert err is None

### Put key-value

In [18]:
writeoptions = crocksdb.writeoptions.create()
key = "key"
value = "value"
err = crocksdb.put(db, writeoptions, key, value)
assert err is None
crocksdb.writeoptions.destroy(writeoptions)

### Read the value back

In [19]:
readoptions = crocksdb.readoptions.create()
rv, err = crocksdb.get(db, readoptions, "key")
assert err is None
assert rv == "value"
crocksdb.readoptions.destroy(readoptions)

TypeError: get() missing 2 required positional arguments: 'rv' and 'rvlen'

### Create a backup in DBBackupPath

In [20]:
err = crocksdb.backup_engine.create_new_backup(be, db)
assert err is None
crocksdb.close(db)

### Restore backup back into DBPath

In [21]:
restore_options = crocksdb.restore_options.create()
err = crocksdb.backup_engine.restore_db_from_latest_backup(
    be, DBPath, DBPath, restore_options
)
assert err is None
crocksdb.restore_options.destroy(restore_options)

### Reopen the restored database

In [22]:
db, err = crocksdb.open(options, DBPath)
assert err is None

### Cleanup

In [23]:
crocksdb.options.destroy(options)
crocksdb.backup_engine.close(be)
crocksdb.close(db)